In [42]:
import requests
import os
import wandb
from dotenv import load_dotenv

In [43]:
load_dotenv()
OPEN_PRICE_API_KEY = os.environ.get('OPEN_PRICE_API_KEY')

We should be able to use any of the fuel types available on open pricing engine since they are all very correlated

In [44]:
headersAuth = {
    'Authorization': f'{OPEN_PRICE_API_KEY}',
}

response = requests.get('https://openpricengine.com/api/v0.2/fuel/price?fuel_type=lrp95coast', headers=headersAuth)

In [45]:
fuel_table = {}
for entry in response.json()['LRP 95 Coast']:
    fuel_table[list(entry.keys())[0]] = list(entry.values())[0]
fuel_df = pd.DataFrame(fuel_table.items(), columns=['date', 'fuel_price'])
fuel_df['date'] = fuel_df['date'].map(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m'))
fuel_df = fuel_df.set_index('date')
fuel_df.to_csv('fuel.csv')

In [46]:
run = wandb.init(
    project="zindi-rmb", 
    job_type='upload-dataset'
    )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: janfaber47 (pluto_is_a_planet). Use `wandb login --relogin` to force relogin


In [48]:
artifact = wandb.Artifact(
    name='fuel.csv', 
    type='dataset'
    ) 

In [49]:
artifact.add_file(local_path='fuel.csv')

In [50]:
run.log_artifact(artifact)

In [51]:
wandb.finish()